In [48]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

In [49]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [50]:
%watermark -v -m -p numpy,pandas,sklearn -g

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

numpy  : 1.19.2
pandas : 1.2.4
sklearn: 0.0

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 20.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit

Git hash: 42e8c3dd9b18060c61bf295d26851026a2fd68eb



In [21]:
df_raw_16 = pd.read_csv('./data/external/zillow-prize-1/properties_2016.csv', low_memory=False)
df_raw_16.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [22]:
df_raw_17 = pd.read_csv('./data/external/zillow-prize-1/properties_2017.csv', low_memory=False)
df_raw_17.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [23]:
df_raw_train_16 = pd.read_csv('./data/external/zillow-prize-1/train_2016_v2.csv', low_memory=False)
df_raw_train_16.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [24]:
df_raw_train_17 = pd.read_csv('./data/external/zillow-prize-1/train_2017.csv', low_memory=False)
df_raw_train_17.head()

,parcelid,logerror,transactiondate
0,14297519,0.025595,2017-01-01
1,17052889,0.055619,2017-01-01
2,14186244,0.005383,2017-01-01
3,12177905,-0.103410,2017-01-01
4,10887214,0.006940,2017-01-01


In [25]:
df_train_2016 = df_raw_train_16.merge(df_raw_16, on='parcelid', how='left')

In [26]:
df_train_2017 = df_raw_train_17.merge(df_raw_17, on='parcelid', how='left')

In [27]:
full_df = pd.concat([df_train_2016, df_train_2017])
full_df

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.168400,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.004000,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.021800,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.005000,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017-09-20,1.0,NaN,NaN,3.0,3.0,NaN,8.0,...,NaN,NaN,265000.0,379000.0,2016.0,114000.0,4685.34,NaN,NaN,6.037132e+13
77609,11000655,0.020615,2017-09-20,NaN,NaN,NaN,2.0,2.0,NaN,6.0,...,NaN,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,NaN,NaN,6.037101e+13
77610,17239384,0.013209,2017-09-21,NaN,NaN,NaN,2.0,4.0,NaN,NaN,...,1.0,NaN,50683.0,67205.0,2016.0,16522.0,1107.48,NaN,NaN,6.111008e+13
77611,12773139,0.037129,2017-09-21,1.0,NaN,NaN,1.0,3.0,NaN,4.0,...,NaN,NaN,32797.0,49546.0,2016.0,16749.0,876.43,NaN,NaN,6.037434e+13


In [28]:
full_df = full_df.drop('parcelid', axis=1)

In [29]:
full_df

,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,0.027600,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,2.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,-0.168400,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,-0.004000,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,3.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,0.021800,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,2.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,-0.005000,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,-0.002245,2017-09-20,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,NaN,NaN,265000.0,379000.0,2016.0,114000.0,4685.34,NaN,NaN,6.037132e+13
77609,0.020615,2017-09-20,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,NaN,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,NaN,NaN,6.037101e+13
77610,0.013209,2017-09-21,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,1.0,NaN,50683.0,67205.0,2016.0,16522.0,1107.48,NaN,NaN,6.111008e+13
77611,0.037129,2017-09-21,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,NaN,NaN,32797.0,49546.0,2016.0,16749.0,876.43,NaN,NaN,6.037434e+13


In [30]:
(full_df.isnull().mean(axis=0)*100).sort_values()

logerror                         0.000000
transactiondate                  0.000000
longitude                        0.020252
propertylandusetypeid            0.020252
rawcensustractandblock           0.020252
regionidcounty                   0.020252
roomcnt                          0.020252
latitude                         0.020252
fips                             0.020252
bedroomcnt                       0.020252
bathroomcnt                      0.020252
assessmentyear                   0.020252
propertycountylandusecode        0.020847
taxvaluedollarcnt                0.021443
landtaxvaluedollarcnt            0.022039
taxamount                        0.026804
regionidzip                      0.070881
structuretaxvaluedollarcnt       0.315091
censustractandblock              0.527733
calculatedfinishedsquarefeet     0.533689
yearbuilt                        0.631373
fullbathcnt                      1.091204
calculatedbathnbr                1.091204
regionidcity                     1

In [31]:
y = full_df['logerror']
X = full_df.drop(['logerror', 'transactiondate', 'propertycountylandusecode', 'propertyzoningdesc'], axis=1)

In [32]:
numeric_cols = X.select_dtypes(include=["float64"]).columns
category_cols = X.select_dtypes(include="object").columns

In [33]:
for col in category_cols:
    print("Unique values of the column {} : {}".format(col, X[col].unique()))
    print("Unique values of the column {} : {}".format(col, X[col].nunique(dropna=True)))

Unique values of the column hashottuborspa : [nan True]
Unique values of the column hashottuborspa : 1
Unique values of the column fireplaceflag : [nan True]
Unique values of the column fireplaceflag : 1
Unique values of the column taxdelinquencyflag : [nan 'Y']
Unique values of the column taxdelinquencyflag : 1


In [34]:
X[['hashottuborspa', 'fireplaceflag']] = X[['hashottuborspa', 'fireplaceflag']].fillna(False)
X['taxdelinquencyflag'] = X['taxdelinquencyflag'].fillna('N')

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (134310, 55)
y_train shape: (134310,)
X_test shape: (33578, 55)
y_test shape: (33578,)


In [40]:
num_imp = SimpleImputer(missing_values=np.nan, strategy="constant")
# Preprocessing for categorical data
cat_imp = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

col_trans = ColumnTransformer(
    transformers=[
        ('numerical', num_imp, numeric_cols),
        ('category', cat_imp, category_cols)
])

model_xgboost = HistGradientBoostingRegressor(random_state=0)

#Pipeline
pipe = Pipeline(steps=[
    ('preprocessor', col_trans),
    ('model', model_xgboost )
])

# Defining parameters for the GridSearchCV
parameters = {
    'model__max_depth': [100, 120, 150, 200],
    'model__learning_rate': [0.02,0.05,0.07]
}

search = GridSearchCV(
    estimator = pipe,
    param_grid = parameters,
    cv = 3
)

# Fit the model
search.fit(X_train, y_train)

print('-----')
print(f'Best parameters {search.best_params_}')
print(
    f'Mean cross-validated accuracy score of the best_estimator: '+ \
    f'{search.best_score_:.3f}'
)
print('-----')

-----
Best parameters {'model__learning_rate': 0.05, 'model__max_depth': 100}
Mean cross-validated accuracy score of the best_estimator: 0.011
-----


In [41]:
print(search.score(X_train, y_train))

0.047790188327193395


In [42]:
y_pred = search.predict(X_test)
y_pred = pd.DataFrame(y_pred)
y_pred.head()

,0
0,0.012044
1,0.018159
2,0.013800
3,-0.031529
4,0.007226


In [51]:
print('Mean absolute Log error: ', round(mean_absolute_error(y_test, y_pred), 3))

Mean absolute Log error:  0.07
